In [3]:
# pip install python-edgar

Note: you may need to restart the kernel to use updated packages.


In [4]:
import edgar
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd

In [12]:
# downloads a tab separated file containing company details and filings index
user_agent = 'prachilad96@gmail.com'
since_year = 2020
edgar.download_index('C:/Users/Admin/Documents/Python Scripts/Edgar', since_year,user_agent, skip_all_present_except_last=False)

In [13]:
csv = pd.read_csv('C:/Users/Admin/Documents/Python Scripts/Edgar/2020-QTR2.tsv',sep='/t', lineterminator='\n', names=None)
csv.columns.values[0]='Item'
selectedreport = '10-K'
company_reports = []

csv.head()

<ipython-input-13-d77db4894f19>:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  csv = pd.read_csv('C:/Users/Admin/Documents/Python Scripts/Edgar/2020-QTR2.tsv',sep='/t', lineterminator='\n', names=None)


,Item
0,1000045|NICHOLAS FINANCIAL INC|4/A|2020-06-09|...
1,1000045|NICHOLAS FINANCIAL INC|4|2020-06-09|ed...
2,1000045|NICHOLAS FINANCIAL INC|4|2020-06-10|ed...
3,1000045|NICHOLAS FINANCIAL INC|4|2020-06-29|ed...
4,1000045|NICHOLAS FINANCIAL INC|8-K|2020-04-03|...


In [14]:
reports = csv[(csv['Item'].str.contains(selectedreport))] # from the df filter reports of selected report
reports.head()


,Item
70,1000209|MEDALLION FINANCIAL CORP|10-K/A|2020-0...
594,1000683|BLONDER TONGUE LABORATORIES INC|10-K/A...
595,1000683|BLONDER TONGUE LABORATORIES INC|10-K|2...
623,1000694|NOVAVAX INC|10-K/A|2020-04-29|edgar/da...
753,1001171|BROADWAY FINANCIAL CORP \DE\|10-K/A|20...


In [15]:
Filing = reports['Item'].str.split('|').to_list()
companies=[]
for i in Filing: # getting only 10-K reports and not 10-K/A
    if i[2] == '10-K':
        companies.append(i)

In [16]:
companies[0]

['1000683',
 'BLONDER TONGUE LABORATORIES INC',
 '10-K',
 '2020-04-13',
 'edgar/data/1000683/0001213900-20-009053.txt',
 'edgar/data/1000683/0001213900-20-009053-index.html']

In [17]:
# getting the html link of the index page of all companies
for item in companies:
#    print(item)
    if 'html' in item[5]:
        url = "https://www.sec.gov/Archives/" + item[5]
        company_reports.append(url)

In [42]:
for i in company_reports[0:5]:
    print(i)

https://www.sec.gov/Archives/edgar/data/1000683/0001213900-20-009053-index.html
https://www.sec.gov/Archives/edgar/data/1001463/0001185185-20-000904-index.html
https://www.sec.gov/Archives/edgar/data/1002047/0001564590-20-029349-index.html
https://www.sec.gov/Archives/edgar/data/1002135/0001002135-20-000020-index.html
https://www.sec.gov/Archives/edgar/data/1004989/0001437749-20-007674-index.html


In [51]:
# from the edgar index page scraping the report link using BeautifulSoup
# the final report link is appended in list reports
# this cell takes time to run
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.164 Safari/537.36'}
reports = []
selectedreport = '10-K'
for i in company_reports:
#     print(i)
    response = requests.get(i,headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    tables=soup.find_all('table')
#     print(tables)
    
    df=pd.read_html(str(tables[0]))
    df = df[0]
#     print(df)
    df = df.dropna()
    document = df[df['Type'].str.contains(selectedreport)]
    try:
        document = document['Document'].str.split(' ')[0][0]
    
        
        final_report_link= i.replace('-', '').replace('index.html','')
        final_url = final_report_link + '/' + document
        reports.append(final_url)
    except IndexError:
        pass
    except KeyError:
        pass

In [52]:
reports[0:5]

['https://www.sec.gov/Archives/edgar/data/1000683/000121390020009053/f10k2019_blondertongue.htm',
 'https://www.sec.gov/Archives/edgar/data/1001463/000118518520000904/acaciadiv20191231_10k.htm',
 'https://www.sec.gov/Archives/edgar/data/1002047/000156459020029349/ntap-10k_20200424.htm',
 'https://www.sec.gov/Archives/edgar/data/1002135/000100213520000020/wstl-2020331x10k.htm',
 'https://www.sec.gov/Archives/edgar/data/1004989/000143774920007674/sgrp20191231_10k.htm']

In [35]:
# The report will be downloaded using the SEC PDF Render API using the parameters token, type and url of report 
# Using the Selenium api for automatically accessing the api that downloads reports on the system

driver = webdriver.Chrome("C:/Users/prachi/.wdm/drivers/chromedriver/win32/91.0.4472.101/chromedriver.exe") #chromedriver path
for i in reports: # using the sec api
    url = 'https://api.sec-api.io/filing-reader?token=c01d386517374a550d167fbe2657f902eb82c4f9f9540a8c4afa73bcb6bfad9a&type=pdf&url=' + i
    driver.get(url)
    